## 18.3 Homework

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
- 1 the average number of attempts to find the ground state
- 2 the time costs



### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution
    write function that get analytic gradient of LJ and put it into the hessien and run the code

In [157]:
import numpy as np
from numba import jit
import pandas as pd

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def dLJdr(r):
    r7 = r**7
    r13 = r**13
    return 24*(1/r7 - 2/r13)

@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)
        
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
    return E

def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

def numeric_grad(f,r):
    eps = 1e-6
    return ((f(r+eps) - f(r))/eps)

def get_true_LJ(N):
    url = "http://doye.chem.ox.ac.uk/jon/structures/LJ/points/"+str(N)
    names = ['x', 'y', 'z']
    dataset = pd.read_csv(url, names=names, delim_whitespace=True)
    pos = dataset.values
    pos = np.reshape(pos, [N*3,1])
    return total_energy(pos)

In [158]:
## from scipy.optimize import minimize
import time

N = [4,8,12,16,20]

_method = ['BFGS',
           'CG',
           'L-BFGS-B']

threshold = 1000
for N_atom in N:
    print('\n --------------------------------------')
    tot_steps = 0
    trueE = get_true_LJ(N_atom)
    for m in _method:
        calculations = []
        print('%s (%s)' %(m, N_atom))
        f_values = 0
        count = 0
        init_time = time.time()
        while(f_values - trueE > 1e-2 and count < threshold):
            pos = init_pos(N_atom)
            res = minimize(total_energy, pos, method=m, tol=1e-4, jac = False)
            f_values = res.fun
            calculations.append(f_values)
            count += 1 
            print('\r Step: {:d} value: {:.4f} Time: {:.2f} '
                  .format(count,res.fun,time.time()-init_time),flush=True,end='') 
        tot_steps += count
        
        print('\n')
    print('\n Avg # of attempts: ', round(tot_steps/3,2))
    print('\n')


 --------------------------------------
BFGS (4)
 Step: 1 value: -6.0000 Time: 0.22 

CG (4)
 Step: 1 value: -6.0000 Time: 0.01 

L-BFGS-B (4)
 Step: 15 value: -6.0000 Time: 0.06 


 Avg # of attempts:  5.67



 --------------------------------------
BFGS (8)
 Step: 2 value: -19.8215 Time: 0.18 

CG (8)
 Step: 1 value: -19.8215 Time: 0.17 

L-BFGS-B (8)
 Step: 5 value: -19.8209 Time: 0.29 


 Avg # of attempts:  2.67



 --------------------------------------
BFGS (12)
 Step: 3 value: -37.9676 Time: 1.20 

CG (12)
 Step: 49 value: -37.9676 Time: 33.00 

L-BFGS-B (12)
 Step: 24 value: -37.9649 Time: 5.51 


 Avg # of attempts:  25.33



 --------------------------------------
BFGS (16)
 Step: 12 value: -56.8157 Time: 11.95 

CG (16)
 Step: 3 value: -56.8157 Time: 5.59 

L-BFGS-B (16)
 Step: 17 value: -56.8109 Time: 10.22 


 Avg # of attempts:  10.67



 --------------------------------------
BFGS (20)
 Step: 277 value: -68.5893 Time: 681.93 

/Users/sayred1/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py:1046: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))


 Step: 312 value: -77.1770 Time: 771.89 

CG (20)
 Step: 204 value: -47.2550 Time: 1781.55 

KeyboardInterrupt: 